In [105]:
import re
import numpy as np

PERCENT_FOR_DEV = .15
PERCENT_FOR_TEST = .1
DIRECTORY='./construction_dataset/'

In [106]:
with open('../tensorflow-yolov3/construction_dataset/dataset.txt','r') as f:
    d = f.readlines()

In [119]:
doubled_d = []

for entry in d:
    
    L1 = re.sub('hardhat','0',entry)
    L2 = re.sub('person','1',L1)
    L3 = re.sub('safetyvest','2',L2)
    final_string = DIRECTORY + L3
    
    item_list = final_string.split(' ')
    metronome = 0
    flipped_list = [re.sub('.jpeg','__fliph.jpeg',item_list[0])]
    for idx,item in enumerate(item_list[1:]):
        if metronome == 0:
            new_xmax = round(640-float(item), 2)
        elif metronome == 1:
            ymin = item
        elif metronome == 2:
            new_xmin = round(640-float(item), 2)
        elif metronome == 3:
            ymax = item
        elif metronome == 4:
            class_id = item
        metronome += 1
        if metronome == 5:
            flipped_list = flipped_list + [str(new_xmin), str(ymin), str(new_xmax), str(ymax), class_id]
            metronome = 0
    flipped_string = " ".join(flipped_list)
    
    doubled_d.append(final_string)
    doubled_d.append(flipped_string)

In [108]:
non_train_idxs = np.random.choice(range(0,len(doubled_d)),int(len(doubled_d)*(PERCENT_FOR_TEST + PERCENT_FOR_DEV)), replace=False)
dev_percent_of_non_train = PERCENT_FOR_DEV / (PERCENT_FOR_DEV + PERCENT_FOR_TEST)
dev_idxs = np.random.choice(non_train_idxs,int(len(non_train_idxs)*dev_percent_of_non_train), replace=False)
dev_idxs_set = set(dev_idxs)
non_train_set = set(non_train_idxs)

In [121]:
train_list = []
dev_list = []
test_list = []

for idx, line in enumerate(doubled_d):
    if idx in non_train_set:
        if idx in dev_idxs_set:
            dev_list.append(line)
        else:
            test_list.append(line)
    else:
        train_list.append(line)

In [123]:
with open('../tensorflow-yolov3/construction_dataset/construction_train.txt','w') as f:
    f.writelines(train_list)

with open('../tensorflow-yolov3/construction_dataset/construction_dev.txt','w') as f:
    f.writelines(test_list)
    
with open('../tensorflow-yolov3/construction_dataset/construction_test.txt','w') as f:
    f.writelines(dev_list)